實作各種不一樣的方式來調整飽和 / 對比 / 明亮

改變 color space 來調整飽和度
實作直方圖均衡
alpha/ beta 調整對比 / 明亮

In [8]:
import cv2
import numpy as np


img = cv2.imread('lena.png', cv2.IMREAD_COLOR)

改變飽和度
轉換成 HSV color space, 改變 s channel 的值

In [9]:
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [13]:
change_percentage = 0.25

img_hsv_down = img_hsv.astype('float32')
img_hsv_down[..., -1] = img_hsv_down[..., -1]/255. - change_percentage
img_hsv_down[img_hsv_down[..., -1] < 0] = 0
img_hsv_down[..., -1] = img_hsv_down[..., -1]*255.
img_hsv_down = img_hsv_down.astype('uint8')

In [26]:
change_percentage = 0.25

img_hsv_up = img_hsv.astype('float32')
img_hsv_up[..., -1] = img_hsv_up[..., -1]/255. + change_percentage
img_hsv_up[img_hsv_up[..., -1] > 1] = 1
img_hsv_up[..., -1] = img_hsv_up[..., -1]*255.
img_hsv_up = img_hsv_up.astype('uint8')

In [27]:
img_hsv_down = cv2.cvtColor(img_hsv_down, cv2.COLOR_HSV2BGR)
img_hsv_up = cv2.cvtColor(img_hsv_up, cv2.COLOR_HSV2BGR)

In [29]:
img_hsv_change = np.hstack((img, img_hsv_down, img_hsv_up))
while True:
    cv2.imshow('change saturation', img_hsv_change)
    
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

case 1: 把彩圖拆開對每個 channel 個別做直方圖均衡再組合起來
case 2: 轉換 color space 到 HSV 之後對其中一個 channel 做直方圖均衡

In [45]:
# case 1
# 每個 channel 個別做直方圖均衡
equalHist_by_blue = cv2.equalizeHist(img[..., 0])
equalHist_by_green = cv2.equalizeHist(img[..., 1])
equalHist_by_red = cv2.equalizeHist(img[..., 2])

# 組合經過直方圖均衡的每個 channel
img_bgr_equal = img
img_bgr_equal[..., 0] = equalHist_by_blue
img_bgr_equal[..., 1] = equalHist_by_green
img_bgr_equal[..., 2] = equalHist_by_red

# case 2 - 轉換 color space 後只對其中一個 channel 做直方圖均衡
img_hsv_equal = img
img_hsv_equal[...,-1] = cv2.equalizeHist(img_hsv[..., -1])

# 組合圖片 + 顯示圖片
img_bgr_equalHist = np.hstack((img, img_bgr_equal, img_hsv_equal))
while True:
    # 比較 (原圖, BGR color space 對每個 channel 做直方圖均衡, HSV color space 對明度做直方圖均衡)
    cv2.imshow('bgr equal histogram', img_bgr_equalHist)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

In [46]:
# alpha: 控制對比度 (1.0~3.0)
# beta: 控制明亮度 (0~255)
add_contrast = cv2.convertScaleAbs(img, alpha = 2.0)
add_lighness = cv2.convertScaleAbs(img, beta = 100)

# 組合圖片 + 顯示圖片
img_contrast_light = np.hstack((img, add_contrast, add_lighness))
while True:
    # 比較不同程度的對比 / 明亮
    cv2.imshow('adjust contrast and brighness', img_contrast_light)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break